# Import the needed libraries

In [6]:

# Install summarytools using pip
%pip install summarytools
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from summarytools import dfSummary

Note: you may need to restart the kernel to use updated packages.


## Import the Data

In [7]:
# Data imports
train_data = pd.read_csv(r'../project_data/train_data.csv')
test_data = pd.read_csv(r'../project_data/test_data.csv')

/var/folders/hr/bvcr3w9n2gxgqdhjgthjjnv80000gn/T/ipykernel_92589/1448665868.py:2: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(r'../project_data/train_data.csv')


### 1.1 Check data contents, provide descriptive statistics and check for incoherencies in the data.

In [8]:
# Check for column names in training data
print(train_data.columns)

# First 5 rows of the training data
train_data.head()

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [9]:
# Check for column names in test data
train_data.columns

# Find column names that don't exist in the test data but exist in the training data
missing_cols = [col for col in train_data.columns if col not in test_data.columns]

print("These will be the columns and the types of the columns that are not in testing data.\n"
    "We will try to predict these columns: \n" + str(missing_cols))


These will be the columns and the types of the columns that are not in testing data.
We will try to predict these columns: 
['Claim Injury Type', 'Agreement Reached', 'WCB Decision']


#### These will be the columns and description of the column from metadata that are not in testing data. These are the columns we are trying to predict: 
 
| Claim Injury Type | Agreement Reached | WCB Decision |
|------------------|-----------------------------------------------------------------|--------------|
|     **Main target variable**: Deliberation of the WCB relative to benefits awarded to the claim. **Numbering indicates severity**.       |     **Binary variable**: Yes if there is an agreement without the involvement of the WCB -> unknown at the start of a claim    |   **Multiclass variable**: Decision of the WCB relative to the claim: “Accident” means that claim refers to workplace accident, “Occupational Disease” means illness from the workplace. -> requires WCB deliberation so it is unknown at start of claim.    |



## Let's take more comprehensive look into data contents and let's make observations

In [10]:
# Check characteristics of the dataframe
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593471 entries, 0 to 593470
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       570337 non-null  object 
 1   Age at Injury                       574026 non-null  float64
 2   Alternative Dispute Resolution      574026 non-null  object 
 3   Assembly Date                       593471 non-null  object 
 4   Attorney/Representative             574026 non-null  object 
 5   Average Weekly Wage                 545375 non-null  float64
 6   Birth Year                          544948 non-null  float64
 7   C-2 Date                            559466 non-null  object 
 8   C-3 Date                            187245 non-null  object 
 9   Carrier Name                        574026 non-null  object 
 10  Carrier Type                        574026 non-null  object 
 11  Claim Identifier          

### Descriptive statistics

In [11]:
# Check summary of the data
dfSummary(train_data)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Accident Date[object],1. nan2. 2020-03-013. 2020-12-184. 2022-02-075. 2022-01-056. 2021-02-187. 2021-02-088. 2020-02-069. 2022-01-3110. 2022-01-2011. other,"23,134 (3.9%)1,245 (0.2%)1,001 (0.2%)977 (0.2%)883 (0.1%)851 (0.1%)850 (0.1%)846 (0.1%)835 (0.1%)818 (0.1%)562,031 (94.7%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAD2UlEQVR4nO3cQWojVxhG0b+McNoTCWHwEpI9aBG92F6EFpE9GKMatB1PlEETCKbdnYFzRZRzZlZN3uDCo6A+L+fzeaBwc+kD8P8hNjJiIyM2MmIjIzYyYiMjNjJiI7PMzN3M3H7n2ev5fH6Oz8MV29zf33/ebrf7tw/WdX1aluWL4Pgom+12uz8cDs+73e7lrx9Pp9On4/G4f3x8vJ0ZsfEhNjMzu93u5eHh4eubZ3cXOA9XzAsCGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGR2cx8+37t7z++/Rs+wmZd16fj8bifN9+vrev6NDOvlzkW18gGgcziX2ZR8YJA5mZZlt+WZfFCwL/uZmZ+nZlfLn0Qrp9rlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzM3M/D4zf1z6IFw/GwQyrlEyYiNj8ELG4IWMa5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzKuUTJiIyM2MtZVZKyryLhGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i4xolIzYyBi9kDF7IuEbJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MgYvJAxeCHjGiUjNjIGL2QMXsi4RsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjHUVGesqMq5RMmIjY11FxrqKjGuUjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2MwQsZgxcyrlEyYiNj8ELG4IWMa5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzKuUTJiIyM2MtZVZKyryLhGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i4xolIzYyBi9kDF7IuEbJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MgYvJAxeCHjGiWzzMzdzNxe+iBchdfz+fz83sPN/f395+12uy9PxHVa1/VpWZYv7wW32W63+8Ph8Lzb7V7qw3E9TqfTp+PxuH98fLydme/HNjOz2+1eHh4evqan4xrd/eihFwQyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYym5lvn4dc+iD8t/2Thjbruj4dj8f9/OTzEPiZdV2fZub1vec+C+cj/fCzcOsqMl4QyIiNjNjIiI2M2MiIjYzYyIiNjNjI/AmX5J400wrMVgAAAABJRU5ErkJggg=="">","23,134(3.9%)"
2,Age at Injury[float64],Mean (sd) : 42.1 (14.3)min < med < max:0.0 < 42.0 < 117.0IQR (CV) : 23.0 (3.0),108 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACiElEQVR4nO3dvW7aUBiH8feUD9kgGaFIsHEDlRgYuYhebC+hA1fAysSGQAgEjYMx7pQqVUlS++Daf+v57Sec4TGJP/LaZVlmQN19qXoDwL9oV72BjzjnQjPrFlx+ybLs+ZH7QXVqG6pzLnx6evoWRdGwyPrj8bh3zn0vGisHSb3UNlQz60ZRNJzP58+DwSDOs/BwOASLxWK42+26ZpY7mKoPEvytzqGamdlgMIhHo9HPAktDj4+t7CDBfbUPtUoVHSS4g7N+SCBUSCBUSCBUSCBUSCBUSGjs5ak0TTtmFjnniiyPbrdb58FbgodGhno+nztJkkwnk0mr1WrlumBvZpYkSZCm6dc4jn+YWZHrqHiwRoZ6uVzaQRD0ZrNZPB6P93nXr9fr4XK57F2v11YZ+0N+jQz1Vb/fL3Rnab/fc2epZjiZggRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChQRChYRG/3NfVTxnCjCt+g5CfTDfmQJMq76PUB/MZ6YA06rfR6glKTpTwJhWfRcnU5BAqJBAqJBAqJBAqJBAqJBAqJBAqJBQ6gV/zzc0M0cfv5UWqu8bmpmjj7fK/EYt/IZmM+bo40+l3+sv+oZm5ujjLU6mIIFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFJKTXjOWDNrKFD1j4N1eMpfZ7Qz8l3wJpZc4esfRiqz1P6PKGfn8+ANbNmD1n77Bu1G0XRcDqdWhiGL3l+8Ha7DVarVe9wOPQ3m80l78ZOp1NgZnY+n4PNZtP7X2ur/OzXtT7SNG2b358OVXnJsuzd3yK/AFkxIPmhibpmAAAAAElFTkSuQmCC"">","19,445(3.3%)"
3,Alternative Dispute Resolution[object],1. N2. nan3. Y4. U,"571,412 (96.3%)19,445 (3.3%)2,609 (0.4%)5 (0.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB+ElEQVR4nO3cMW7acBjG4c/ISpvFFrKUI7R34BA5bA/hQ/QOCOGhSbPQIelQApEq6teS+zyjp//wk8HIL83pdCpI2Cx9AP4fYiOmqar7qrpb+iCsysvpdHo6v9gOw/DYdd12iROxTtM0HZqm+XYeXNt13Xa32z31ff+81OFYj+Px+Hkcx+1+v7+rqj9jq6rq+/754eHhxyKnY43uL130gECM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxLRVr6+FLH0Q1uGjltppmg7jOG7rymsh8LemaTpU1cv5da+FM4eLr4U3pnykeEAg5trH6MXbINzi4rrq2joGbvFuXfXROgZucW1d5WcQ/jkPCMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiIeTflM+tjLhenfNemWHALGwRiTPmI+X1nK3cy5tYOw/BYVWVNxdw2Xddt36Z8/oKBWfmdjRixESM2YsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxY

In [12]:
# Check the descriptive statistics for non-object values
train_data.describe(exclude='object')

,Age at Injury,Average Weekly Wage,Birth Year,Claim Identifier,IME-4 Count,Industry Code,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Agreement Reached,Number of Dependents
count,574026.000000,5.453750e+05,544948.000000,5.934710e+05,132803.000000,564068.000000,0.0,558386.000000,558369.000000,556944.000000,574026.000000,574026.000000
mean,42.114270,4.910883e+02,1886.767604,2.366760e+07,3.207337,58.645305,NaN,54.381143,41.013839,39.738146,0.046665,3.006559
std,14.256432,6.092918e+03,414.644423,1.079271e+08,2.832303,19.644175,NaN,25.874281,22.207521,22.365940,0.210921,2.000801
min,0.000000,0.000000e+00,0.000000,5.393066e+06,1.000000,11.000000,NaN,1.000000,1.000000,-9.000000,0.000000,0.000000
25%,31.000000,0.000000e+00,1965.000000,5.593414e+06,1.000000,45.000000,NaN,31.000000,16.000000,33.000000,0.000000,1.000000
50%,42.000000,0.000000e+00,1977.000000,5.791212e+06,2.000000,61.000000,NaN,56.000000,49.000000,38.000000,0.000000,3.000000
75%,54.000000,8.410000e+02,1989.000000,5.991000e+06,4.000000,71.000000,NaN,75.000000,52.000000,53.000000,0.000000,5.000000
max,117.000000,2.828079e+06,2018.000000,9.998917e+08,73.000000,92.000000,NaN,99.000000,91.000000,99.000000,1.000000,6.000000


In [15]:
# Check the descriptive statistics for object values
train_data.describe(include='object')

,Accident Date,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,...,District Name,First Hearing Date,Gender,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Description,WCIO Nature of Injury Description,WCIO Part Of Body Description,Zip Code,WCB Decision
count,570337,574026,593471,574026,559466,187245,574026,574026,574026,574026,...,574026,150798,574026,564068,574026,558386,558369,556944,545389,574026
unique,5539,3,1096,2,2475,1648,2046,8,8,63,...,8,1094,4,20,5,74,56,54,10060,1
top,2020-03-01,N,2020-03-06,N,2021-05-11,2021-04-21,STATE INSURANCE FUND,1A. PRIVATE,2. NON-COMP,SUFFOLK,...,NYC,2022-04-15,M,HEALTH CARE AND SOCIAL ASSISTANCE,IV,LIFTING,STRAIN OR TEAR,LOWER BACK AREA,11236,Not Work Related
freq,1245,571412,1422,392291,1847,350,111144,285368,291078,60430,...,270779,440,335218,114339,265981,46610,153373,51862,3302,574026


## Notes and incoherensies:
* SOMETHING HERE

## Visualisations of data and study of relationships

## Conclusions